# 第一章 数据来源

对于机器学习来说，数据是至关重要的，没有数据，机器学习也无从谈起，因此，第一步我们将讨论数据来源。  
本章将主要讨论3个数据来源：文件、数据库和API。  

## 数据源

### 文件

文件是常见的数据来源之一，例如Excel文件、csv文件等。  
`pandas`库提供了读取多种常用文件类型的方法，例如

- `pandas.read_csv`：读取csv文件，可以通过设置`sep`可以读取不同分隔符的文件，例如`tab`为分割符的文件
- `pandas.read_excel`：读取Excel文件（需要安装`openpyxl`库）
- `pandas.read_json`：读取JSON文件
- `pandas.read_sas`：读取SAS文件
- `pandas.read_ssps`：读取SPSS文件

In [1]:
# 利用pandas.read_csv方法来读取一个csv文件

import pandas as pd

df = pd.read_csv('data/nba_players.csv')
df.head()

,player_id,player_name,avg_pts,avg_ast,avg_oreb,avg_dreb,avg_stl,avg_blk,avg_tov,avg_fgm,avg_fga,avg_tpm,avg_play_time,team_name,position
0,1,凯尔-科沃尔,10.12,1.64,0.13,2.66,0.49,0.31,1.03,3.57,7.66,2.42,26.13,骑士,2.5
1,2,蒂亚戈-斯普利特,4.33,0.44,0.89,1.56,0.11,0.11,0.67,1.56,3.44,0.22,8.56,76人,5.0
2,3,保罗-米尔萨普,17.87,3.63,1.59,6.13,1.30,0.87,2.29,6.17,13.94,1.09,33.66,老鹰,4.0
3,4,萨博-塞福洛沙,7.16,1.73,0.84,3.52,1.48,0.50,0.95,2.81,6.35,0.66,25.73,老鹰,2.5
4,5,杰夫-蒂格,15.29,7.78,0.39,3.66,1.24,0.39,2.63,4.90,11.07,1.10,32.44,步行者,1.0


此外，`pandas`还提供了一个`read_clipboard`方法，可以方便读取你粘贴板中的内容，例如我们复制以下内容:  

>
date,h,l,w  
2020-01-01,19,10,rainy  
2020-01-02,18,11,sunny  
2020-01-03,17,10,sunny  
>

In [2]:
# 读取粘贴板中的内容

pd.read_clipboard(sep=',')

,date,h,l,w
0,2020-01-01,19,10,rainy
1,2020-01-02,18,11,sunny
2,2020-01-03,17,10,sunny


这样，我们可以方便的复制Excel中的数据，然后使用`read_clipboard`方法，可以快速读取数据  
例如我们复制下图中Excel的数据

![excel_screenshot](resource/chp1_excel_clipboard.png)

In [3]:
# 读取粘贴板中Excel表中的内容

pd.read_clipboard(sep='\t')

,id,name,score
0,1,Marry,8.2
1,2,Peter,9.1
2,3,Jhon,8.6


与读取相对，`pandas`库也提供了一系列写入不同文件格式的方法。一般一个`DataFrame`中有`to_*`的方法可以用于将数据输出到一个文件中，如`to_csv`，`to_excel`等。  
例如我们将之前从粘贴板中的内容再写入到一个csv文件中

In [4]:
# 将粘贴板中内容再写入一个csv文件中

# date,h,l,w
# 2020-01-01,19,10,rainy
# 2020-01-02,18,11,sunny
# 2020-01-03,17,10,sunny

df = pd.read_clipboard(sep=',')
# 将index设为False表示不输出index
df.to_csv('data/excel_content.csv', index=False)

### 数据库

数据库是数据另一个数据主要来源，特别在工程环境下，大部分数据都来源于数据库。`pandas`库中提供了`read_sql`方法，可以通过sql语句来读取数据库中的数据，这是我们最常用的读取数据库中数据并转换成`DataFrame`的方法。  
`read_sql`需要设置两个必要参数，`sql`和`con`

- `sql`：sql语句  
- `con`：数据库的链接  
    创建获取数据库链接，我们可以使用数据库相应的库来完成，例如`sqlite3`、`pymysql`等  

例如，我们从本地的`MySQL`数据库读取`user`表中的数据

~~~Python
import pymysql
import pandas as pd

# 创建MySQL链接
conn = pymysql.connect(
    host='127.0.0.1',   # MySQL地址
    port=3306,          # MySQL端口
    user='root',        # MySQL用户名
    password='123456',  # MySQL密码
    database='test'     # MySQL数据库
)
sql = 'select * from user'
data = pd.read_sql(sql, conn)
# 关闭MySQL链接
conn.close() 
~~~

此外还有一个比较常用的参数是`parse_dates`，设置这个参数可以将指定的列转化为日期类型。

In [5]:
# 读取SQLite库中的user表中的全部数据

import sqlite3

# SQLite数据在data文件夹下
conn = sqlite3.connect('data/sqlite_db.db')
# 使用read_sql方法读取数据库
df = pd.read_sql('SELECT * FROM user', conn)
conn.close()

df.head()

,id,name,score,level
0,1,Mary,9.3,A
1,2,Peter,8.1,B
2,3,Jhon,8.2,B
3,4,Jane,7.4,C
4,5,James,8.1,B


同样与`read_sql`读取数据库数据相对的是`DataFrame`中提供了`to_sql`方法，用于写入数据库。  

In [7]:
# 将数据写入SQLite数据库

# 构造一个写入的数据
df = pd.DataFrame({'name': ['James', 'Steven'], 
                   'score': [8.1, 9.3], 
                   'level': ['B', 'A']})
conn = sqlite3.connect('data/sqlite_db.db')
# 使用to_csv方法写入到数据库中
df.to_sql('user', conn, if_exists='append', index=False)
conn.close()

在`to_sql`方法，有个比较常用的参数：`if_exists`，其表示当遇到表已经存在时，我们应当怎么做，他有3个可选的参数

- `fail`：当表存在时，抛出异常，这也是默认选项
- `replace`：丢弃当前的表，重新建一个新表，随后插入数据
- `append`：追加模式，插入数据到已经存在的表中

其中，`append`是比较常用的一种方式，因此使用这个方法时，时常记住看一下这个参数是否被设置正确。  
还有有个需要注意的参数就是`index`，默认的为`True`，表示将`index`也插入到数据库中，我们一般会设为`False`，选择不讲`index`插入到数据库中，因此使用时也应当注意这个参。

### API

调用API获取数据也是数据重要的来源之一，例如事实的股票数据，天气数据等，可以通过调用提供这些查询服务的API来获取。  
很多公司内部系统也会通过API形式来提供数据，这样做可以使得内部系统耦合度降低。假设A公司的平台系统中有许多子系统组成，其中用户平台系统提供用户信息API，其他子系统可以调用该API获取用户信息数据，当用户平台内部的需要做一些修改时，例如对数据库的增减进行字段，只要保持API请求参数和响应参数不变，则不会影响其他子系统调用用户信息数据。  
`requests`库提供了一些常用的请求接口的方法，现在大部分API返回的都是`JSON`数据格式，因此可以使用内置的`json`库来方便的处理这些返回数据。  
例如下面是一个通过使用`requests`库从天气查询API中获取天气数据，然后使用`json`库对返回数据进行处理，最后转为一个`DataFrame`

In [8]:
# 从API获取数据

import json
import requests

# 调用API获取上海的天气数据
response = requests.get('http://wthrcdn.etouch.cn/weather_mini?city=上海')
# 使用json.loads函数将json格式数据转为dict
content = json.loads(response.content)
content

{'data': {'city': '上海',
  'forecast': [{'date': '13日星期一',
    'fengli': '<![CDATA[<3级]]>',
    'fengxiang': '东北风',
    'high': '高温 9℃',
    'low': '低温 4℃',
    'type': '阴'},
   {'date': '14日星期二',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '西北风',
    'high': '高温 9℃',
    'low': '低温 4℃',
    'type': '多云'},
   {'date': '15日星期三',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '东北风',
    'high': '高温 10℃',
    'low': '低温 5℃',
    'type': '小雨'},
   {'date': '16日星期四',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '东北风',
    'high': '高温 9℃',
    'low': '低温 7℃',
    'type': '小雨'},
   {'date': '17日星期五',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '西北风',
    'high': '高温 9℃',
    'low': '低温 4℃',
    'type': '阴'}],
  'ganmao': '天气较凉，较易发生感冒，请适当增加衣服。体质较弱的朋友尤其应该注意防护。',
  'wendu': '8',
  'yesterday': {'date': '12日星期日',
   'fl': '<![CDATA[3-4级]]>',
   'fx': '北风',
   'high': '高温 9℃',
   'low': '低温 3℃',
   'type': '阴'}},
 'desc': 'OK',
 'status': 1000}

In [9]:
# 获取天气预报的信息

import re

forecast = content['data']['forecast']
df = pd.DataFrame(forecast)
# 对形如<![CDATA[3-4级]]>风力数据进行处理，保留其中的级数
rex = re.compile('[0-9]*[<-][0-9]*级')
df['fengli'] = df['fengli'].map(lambda x: rex.findall(x)[0])
# 处理高温和低温的字样
df['high'] = df['high'].map(lambda x: x.replace('高温 ', ''))
df['low'] = df['low'].map(lambda x: x.replace('低温 ', ''))
df

,date,high,fengli,low,fengxiang,type
0,13日星期一,9℃,<3级,4℃,东北风,阴
1,14日星期二,9℃,3-4级,4℃,西北风,多云
2,15日星期三,10℃,3-4级,5℃,东北风,小雨
3,16日星期四,9℃,3-4级,7℃,东北风,小雨
4,17日星期五,9℃,3-4级,4℃,西北风,阴


我们也可以同时获取多个城市的预报信息（这里我们可以先封装一个获取单个城市天气的方法，然后调用改方法来完成该功能，这样代码不但简洁，且利于维护，我们将在`Landing`小节中详细展开）

In [10]:
# 获取多个城市的天气预报信息

city_list = ['上海', '北京']
df_list = []

for c in city_list:
    url = f'http://wthrcdn.etouch.cn/weather_mini?city={c}'
    response = requests.get(url)
    content = json.loads(response.content)
    df = pd.DataFrame(content['data']['forecast'])
    df['city'] = c
    rex = re.compile('[0-9]*[<-][0-9]*级')
    df['fengli'] = df['fengli'].map(lambda x: rex.findall(x)[0])
    df['high'] = df['high'].map(lambda x: x.replace('高温 ', ''))
    df['low'] = df['low'].map(lambda x: x.replace('低温 ', ''))
    df_list.append(df)

pd.concat(df_list)

,date,high,fengli,low,fengxiang,type,city
0,13日星期一,9℃,<3级,4℃,东北风,阴,上海
1,14日星期二,9℃,3-4级,4℃,西北风,多云,上海
2,15日星期三,10℃,3-4级,5℃,东北风,小雨,上海
3,16日星期四,9℃,3-4级,7℃,东北风,小雨,上海
4,17日星期五,9℃,3-4级,4℃,西北风,阴,上海
0,13日星期一,2℃,3-4级,-7℃,北风,晴,北京
1,14日星期二,2℃,<3级,-9℃,东北风,晴,北京
2,15日星期三,3℃,<3级,-6℃,西南风,晴,北京
3,16日星期四,3℃,<3级,-5℃,西南风,多云,北京
4,17日星期五,4℃,<3级,-6℃,西南风,多云,北京


## Landing

本章`Landing`小节将会带来2个实战部分：

- API：我们将封装天气数据API，每次我们输入一个城市列表，输出这些城市的天气预报
- 数据库：数据库部分我们将封装一个SQLite类，方便我们操作`SQLite`数据库

对我们常用的一些获取数据的方法进行封装，可以极大的方便我们重用这些代码。同时，当我们对方法进行修改，也能使我们尽少的影响以西其他已完成的代码。使整体的代码逻辑变得更为清晰，利于代码维护。

### API封装

将一些常用的操作写在一个方法里面，是一种常用的封装操作，这样可以方便的复用经常使用的代码块。  
本例中，我们将调用天气数据API获取天气预报数据的代码块封装成一个方法，该方法的目标是输入一个城市列表，如`['上海', '北京', '广州']`，输出该列表上城市的天气预报数据。

我们将该封装的方法写在一个python文件中作为一个单独python的模块。在这个模块中，我们将按照以下三个部分来组织我们的代码。  

1. 引入所需要的库。一般模块中所用到的库，我们都在写文件开头，并且分为3个部分：
    - 系统自带的库
    - 第三方库
    - 自己编写的库/模块等
   

2. 一些可配置的常量。例如API的URL，数据库的用户名、数据库地址等，可以写在这部分。配置这些常量，将会有几个好处：
    - 如果文件中有多个地方需要使用到该配置，那么我们只需要修改这个变量就可以了，不容易出现修改错误、遗漏修改等问题
    - 当我们需要修改某些常量时，在文件头部位置就能找到这些变量并且进行修改，非常方便


3. 功能实现代码。

第一部分我们引入需要使用到的库

In [11]:
# 引入需要的库

import json
import re

import pandas as pd
import requests

第二部分我们配置一些常量

In [12]:
# 配置API地址
URL = 'http://wthrcdn.etouch.cn/weather_mini'
# 配置处理风力的正则表达式
FENGLI_REX = re.compile('[0-9]*[<-][0-9]*级')

第三部分则是我们的功能实现的代码。一般开发遵循的原则是每个方法实现一个单一简单的功能，通过对这些方法聚合，形成一些复杂的功能。  
在本例中，我们可以将查询单个城市的天气预报数据的功能封装成一个方法，然后可以基于这个方法来实现多个城市的查询。

我们先实现一个获取单个城市天气预报的函数`get_city_forecast`

In [13]:
def get_city_forecast(city):
    """
    获取城市的天气预报
    :param city: 城市
    :return: 该城市的天气预报
    """
    response = requests.get(URL, params={'city': city})
    content = json.loads(response.content)
    df = pd.DataFrame(content['data']['forecast'])
    df['fengli'] = df['fengli'].map(lambda x: FENGLI_REX.findall(x)[0])
    df['high'] = df['high'].map(lambda x: x.replace('高温 ', ''))
    df['low'] = df['low'].map(lambda x: x.replace('低温 ', ''))
    df['city'] = city
    
    return df

In [14]:
get_city_forecast('上海')

,date,high,fengli,low,fengxiang,type,city
0,13日星期一,9℃,<3级,4℃,东北风,阴,上海
1,14日星期二,9℃,3-4级,4℃,西北风,多云,上海
2,15日星期三,10℃,3-4级,5℃,东北风,小雨,上海
3,16日星期四,9℃,3-4级,7℃,东北风,小雨,上海
4,17日星期五,9℃,3-4级,4℃,西北风,阴,上海


我们基于`get_city_forecast`方法来实现我们的目标方法

In [15]:
def get_city_forecasts(city_list):
    """
    获取列表中城市的天气预报
    :param city: 城市列表
    :return: 天气预报
    """
    df_list = []
    for city in city_list:
        df = get_city_forecast(city)
        df_list.append(df)
        
    return pd.concat(df_list)

In [16]:
get_city_forecasts(['上海', '北京'])

,date,high,fengli,low,fengxiang,type,city
0,13日星期一,9℃,<3级,4℃,东北风,阴,上海
1,14日星期二,9℃,3-4级,4℃,西北风,多云,上海
2,15日星期三,10℃,3-4级,5℃,东北风,小雨,上海
3,16日星期四,9℃,3-4级,7℃,东北风,小雨,上海
4,17日星期五,9℃,3-4级,4℃,西北风,阴,上海
0,13日星期一,2℃,3-4级,-7℃,北风,晴,北京
1,14日星期二,2℃,<3级,-9℃,东北风,晴,北京
2,15日星期三,3℃,<3级,-6℃,西南风,晴,北京
3,16日星期四,3℃,<3级,-5℃,西南风,多云,北京
4,17日星期五,4℃,<3级,-6℃,西南风,多云,北京


具体的代码实现可以参考`code/chp1/weather_forecast.py`。

封装好API后，我们可以在其他地方引入这个模块，使用其中的方法
~~~python
from weather_forecast import get_city_forecasts


def fun():
    ...
    forecasts = get_city_forecasts(['上海', '北京', '广州'])
    # 操作forecasts
    ...
~~~

当我们做出一些修改时，例如改动API的URL，给方法增加多线程实现时，其他调用这个模块的地方就无需更改任何地方。  
这小节给出的代码实现，相比之前在`API`小节中的实现，代码更为清晰，简洁，有利于我们对于代码进行维护。

### 数据库封装

这一小节我们将封装一个SQLite的类，方便我们操作SQLite数据库。封装MySQL、PostgreSQL可以使用类似的方式。  
这个类中，我们将会实现从数据库读取数据（`load_data`）和向数据库中写入数据（`write_data`）这两个方法。

类的初始化方法中，可以将SQLite文件路径作为类初始化的参数。类似的，MySQL可以将用户名、密码、端口等参数作为初始化的参数。

In [17]:
class SQLiteProcessor:
    def __init__(self, file_path):
        """

        :param file_path: SQLite文件路径
        """
        self.file_path = file_path
    
    ...

与数据库打交道，则离不开与数据库建立链接，因此我们会实现一个私有方法，获取数据库链接。

In [18]:
class SQLiteProcessor:
    ...

    def __get_connect(self):
        """
        获取SQLite数据库的链接
        :return:
        """
        return sqlite3.connect(self.file_path)
    
    ...

接下来我们先实现读取数据的方法。我们将会使用`try-catch`的代码结构，这样做的好处是当调用中发生错误时，我们可以正确关闭数据库的链接，并且捕捉到相应的错误信息，便于排查问题。  
`load_data`方法中除了`sql`参数，我们还加了一个`parse_dates`参数，用于指定哪几列应当转为日期类型。

In [19]:
class SQLiteProcessor:
    ...

    def load_data(self, sql, parse_dates=None):
        """
        读取数据
        :param sql: sql语句
        :param parse_dates: 需要转为日期类型的列
        :return:
        """
        conn = None
        # 使用try-catch结构，当发生错误时，保证数据库链接能被正确关闭
        try:
            conn = self.__get_connect()
            return pd.read_sql(conn, sql, parse_dates=parse_dates)
        except sqlite3.Error as e:
            print('SQLite error@{}'.format(e))
        except Exception as e:
            print('Unknown error@{}'.format(e))
        finally:
            # 关闭数据库链接
            conn.close()

    ...

然后我们实现写入的数据的方法。`write_data`方法与`load_data`非常类似，可以参照`load_data`的实现思路。

In [20]:
class SQLiteProcessor:
    ...

    def write_data(self, table_name, df, if_exists='append', is_index=False):
        """

        :param table_name: 数据要插入到的表名称
        :param df: DataFrame
        :param if_exists: to_sql中的if_exists参数，这里我们默认为append
        :param is_index: to_sql中的index参数，这里我们默认为False
        :return:
        """
        conn = None
        try:
            conn = self.__get_connect()
            df.to_sql(table_name, df, if_exists=if_exists, index=is_index)
        except sqlite3.Error as e:
            print('SQLite error@{}'.format(e))
        except Exception as e:
            print('Unknown error@{}'.format(e))
        finally:
            conn.close()

    ...

最后，我们可以在这个模块中，实例化对象，例如

~~~python
user_sqlite = SQLiteProcessor('data/user.db')
car_sqlite = SQLiteProcessor('data/car.db')
~~~

这样在其他地方需要在引入数据库模块时，如果多个地方需要使用到，例如`user`的数据库，那么只需要引入相应的实例化对象，而不必在多个地方实例化多次。  
完整的代码实现可以参考`code/chp1/db_utils.py`。 

封装完成后，我们就可以在其他地方引入模块中的user_sqlite，读取其中的数据，对数据做操作，将数据写入数据库等操作。

~~~python
from db_utils import user_sqlite

def clean_data():
    ...
    df = user_sqlite.load_sql('select * from data', parse_dates=['stats_date'])
    # 对df做数据清洗等操作
    ...
    user_sqlite.write_data('data_cleaning', df)
    ...
~~~

## 小结

本章节我们主要对三种常用的数据来源，文件、数据库和API，进行了讨论。`pandas`库提供了一些常用的文件和数据库读写操作，`requests`库则方便我们对API进行调用。  
在`Landing`小节中，我们讨论了2个实战内容：API和数据库的封装。封装好常用的一些数据获取方法，对于我们之后的操作，无论是优化获取方法，维护数据源等，都会带来很大的便利。